In [ ]:
#import itertools
#import random
import os

import sys
sys.path.insert(0,'..')

import matplotlib 
import matplotlib.pyplot as plt
%matplotlib inline
#import numpy as np
#import scipy.signal as signal
#import pandas as pd

#import datetime

#import backtrader as bt

#from btgym import BTgymEnv, BTgymStrategy, BTgymDataset

-------------
#### Basic settings and parameters
EBTgymEnv class comes preconfigured for quick setting. Basicaly one need to provide at least data file to set it up. Here all parameters are set to defaults values. One can eyeball env. parameters by looking at nested  `MyEnvironment.params` dictionary consisting of three subdictionaries:
- `params_dataset`,  
- `params_engine`,
- `params_other`

One should look at source files for exact parameters descriptions, since complete reference is not ready yet. :/


In [4]:
from btgym import BTgymEnv
 
MyEnvironment = BTgymEnv(filename='./data/DAT_ASCII_EURUSD_M1_2016.csv',)

# Print parameters and default values:

for params_name, params_dict in MyEnvironment.params.items():
    print('\nParameters [{}] subset: '.format(params_name))
    for key, value in params_dict.items():
        print('{} : {}'.format(key,value))
            


Parameters [dataset] subset: 
filename : ./data/DAT_ASCII_EURUSD_M1_2016.csv
start_weekdays : [0, 1, 2]
start_00 : True
episode_len_days : 1
episode_len_hours : 23
episode_len_minutes : 55
time_gap_days : 0
time_gap_hours : 5
sep : ;
header : 0
index_col : 0
parse_dates : True
names : ['open', 'high', 'low', 'close', 'volume']
timeframe : 1
datetime : 0
open : 1
high : 2
low : 3
close : 4
volume : -1
openinterest : -1

Parameters [engine] subset: 
state_dim_time : 10
state_dim_0 : 4
state_low : 1.03522
state_high : 1.1616
start_cash : 10.0
broker_commission : 0.001
fixed_stake : 10
drawdown_call : 90

Parameters [other] subset: 
portfolio_actions : ('hold', 'buy', 'sell', 'close')
port : 5500
verbose : 0


----
#### Specifying more controls:
There is example of common setup.
 
There's some more parameters, especially at dataset section. Those are related to correct CSV data parsing and better left default until you pick different data source. See BTgymDataset() class source for explanations.

In [5]:
MyEnvironment = BTgymEnv(filename='../examples/data/DAT_ASCII_EURUSD_M1_2016.csv',
                         
                     # Dataset and single random episode related parameters:
                         
                         # We start trading on mondays, thuesdays and wednesdays:
                         start_weekdays=[0, 1, 2],
                         # Want total episode duration to be no more than 1 day 23h 55min:
                         episode_len_days=1,
                         episode_len_hours=23,
                         episode_len_minutes=55,
                         # Want to start every episode at the begiining of the day:
                         start_00=True,
                         
                     # Broker and trade realted:
                         
                         # Set initial capital:
                         start_cash=100,
                         # Set broker commission as 0.1% of operation value:
                         broker_commission=0.002,
                         # We use fixed stake of size 10:
                         fixed_stake=10,
                         # We want stop episode if 50% of initial capital is lost:
                         drawdown_call=50,
                         
                     # RL environment related parameters:
                         
                         # Set time embedding dimensionality;
                         # that's basically means we get sequence of last 20 price values
                         # as space observation: 
                         state_dim_time=30,
                         
                         # Observations min and max. possible values.
                         # Setting these to None forces environment to infer from entire dataset.
                         # Since our obs. space simply consisits of series of prices (by default) - it's reasonable.
                         state_low=None,
                         state_high=None,
                         
                     # Other parameters:
                         
                         # Network port to use:
                         port=5555,
                         # Be chatty: settting this to 1 makes environment report what's going on;
                         # 2 is for debugging, dumps out a lot of data:
                         verbose=1,)


[2017-06-21 19:22:14,290] Using base BTgymDataset class.
[2017-06-21 19:22:14,291] Using base BTgymStrategy class.
[2017-06-21 19:22:15,220] Sucsessfuly loaded 372678 records from <../examples/data/DAT_ASCII_EURUSD_M1_2016.csv>.
[2017-06-21 19:22:15,303] Dataset summary statistic:
                open           high            low          close    volume
count  372678.000000  372678.000000  372678.000000  372678.000000  372678.0
mean        1.107109       1.107198       1.107019       1.107108       0.0
std         0.024843       0.024840       0.024847       0.024844       0.0
min         1.035250       1.035470       1.035220       1.035220       0.0
25%         1.092140       1.092230       1.092040       1.092140       0.0
50%         1.113530       1.113610       1.113450       1.113530       0.0
75%         1.124710       1.124780       1.124630       1.124710       0.0
max         1.161440       1.161600       1.160770       1.161450       0.0
[2017-06-21 19:22:15,303] Data flu

----
#### Registering environment:
OpenaAI way of making environment is to register it with cpecific set of parameters under some unique name and instantiate it via calling standart `make()` method. This helps for standartization and correct evaluation of results uploaded to Gym board. 
 
That's how you do it (same parameters as above):

In [8]:
import gym

env_params = dict(filename='../examples/data/DAT_ASCII_EURUSD_M1_2016.csv',
                  start_weekdays=[0, 1, 2],
                  episode_len_days=1,
                  episode_len_hours=23,
                  episode_len_minutes=55,
                  start_00=True,
                  start_cash=100,
                  broker_commission=0.002,
                  fixed_stake=10,
                  drawdown_call=50,
                  state_dim_time=30,
                  state_low=None,
                  state_high=None,
                  port=5555,
                  verbose=1,)
                  
gym.envs.register(id='backtrader-v46',
                  entry_point='btgym:BTgymEnv',
                  kwargs=env_params)
                  
MyEnvironment = gym.make('backtrader-v46')

[2017-06-21 19:26:09,969] Making new env: backtrader-v46
[2017-06-21 19:26:09,986] Using base BTgymDataset class.
[2017-06-21 19:26:09,987] Using base BTgymStrategy class.
[2017-06-21 19:26:10,772] Sucsessfuly loaded 372678 records from <../examples/data/DAT_ASCII_EURUSD_M1_2016.csv>.
[2017-06-21 19:26:10,860] Dataset summary statistic:
                open           high            low          close    volume
count  372678.000000  372678.000000  372678.000000  372678.000000  372678.0
mean        1.107109       1.107198       1.107019       1.107108       0.0
std         0.024843       0.024840       0.024847       0.024844       0.0
min         1.035250       1.035470       1.035220       1.035220       0.0
25%         1.092140       1.092230       1.092040       1.092140       0.0
50%         1.113530       1.113610       1.113450       1.113530       0.0
75%         1.124710       1.124780       1.124630       1.124710       0.0
max         1.161440       1.161600       1.160770   